# Importing libraries

In [1]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine

# Changing the way pandas displays floats

# Importing dataset

In [2]:
suicides = pd.read_csv('/Users/andressalomferrer/Desktop/ironhack/Projects/Project_3/Project-Week-3-Data-Thieves/your-project/suicides_cleaned')

FileNotFoundError: File b'/Users/andressalomferrer/Desktop/ironhack/Projects/Project_3/Project-Week-3-Data-Thieves/your-project/suicides_cleaned' does not exist

In [ ]:
suicides['Population'].sum()

In [ ]:
suicides.Year.unique()

# QUESTION 4:

# 1. MALES OR FEMALES COMMIT SUICIDE MORE?

In [ ]:
suicides.dtypes

In [ ]:
suicides = suicides.astype({"Year": 'object'})

In [ ]:
# From 1990 till 2015

In [ ]:
tot_s_mf = suicides[['Gender','Num_Suicides','Year']].groupby('Gender').sum().reset_index()

In [ ]:
tot_s_mf['Percentage'] = tot_s_mf['Num_Suicides'].apply(lambda row: (row/sum(tot_s_mf['Num_Suicides'])*100))

In [ ]:
tot_s_mf = tot_s_mf.round({'Percentage': 2})

## 1. Solution: males  77.3%, females: 22.7%

In [ ]:
tot_s_mf

# 2. Males or females commit suicide more in 2015?

In [ ]:
# To answer this question we are going to create a dataframe for the year 2015:

suicides_2015 = suicides[suicides['Year'] == 2015]

In [ ]:
# Now we are going to filter by the columns we need:

s_gender2015 = suicides_2015[['Gender','Num_Suicides']].groupby('Gender').sum().reset_index()

## As we can see, the number of suicides in 2015 is higher in males than in females:


In [ ]:
s_gender2015

In [ ]:
s_gender2015['Percentage'] = s_gender2015['Num_Suicides'].apply(lambda row: (row/sum(s_gender2015['Num_Suicides'])*100))

In [ ]:
s_gender2015

In [ ]:
s_gender2015 = s_gender2015.round({'Percentage': 2})

# 2. Solution: males 76.8%, females 23.2%

In [ ]:
s_gender2015

# 3. TOP 10 COUNTRIES BY GENRE:

## BY NUMBER:

### Top 10 countries with more suicides in year 2015 taking into account the number of suicides.

In [ ]:
top10countriesbynum = suicides_2015[['Country','Num_Suicides']].groupby('Country').sum().sort_values("Num_Suicides", ascending=False)[0:10]

In [ ]:
top10countriesbynum

### MALES : TOP 10 COUNTRIES BY NUM

In [ ]:
top10malesbynum = suicides_2015[['Country','Gender','Num_Suicides']].loc[suicides_2015['Gender']=='male'].groupby('Country').sum().sort_values("Num_Suicides", ascending=False)[0:10]

In [ ]:
top10malesbynum

### FEMALES : TOP 10 COUNTRIES BY NUM

In [ ]:
top10femalesbynum = suicides_2015[['Country','Gender','Num_Suicides']].loc[suicides_2015['Gender']=='female'].groupby('Country').sum().sort_values("Num_Suicides", ascending=False)[0:10]

In [ ]:
top10femalesbynum

In [ ]:
# Creating an intersection by index to see if the countries with a higher number of suicides
# are the same countries for both genres.

idgnf = pd.Index(top10femalesbynum.index)
idgnm = pd.Index(top10malesbynum.index)

In [ ]:
set_intersection = idgnf.intersection(idgnm)

In [ ]:
set_intersection

### As we can see 9 out of 10 countries match

In [ ]:
# Spain appears in top10 of countries where females commit suicides.

idgnf.difference(idgnm)

In [ ]:
# Poland appears in top10 of countries where males commit suicides.

idgnm.difference(idgnf)

In [ ]:
set_intersection = idgnf.intersection(idgnm)

In [ ]:
set_intersection

## Now we realize that took into account the num of suicides. But if we take into account the size of the population, results can be different. Let's check:


## BY SIZE:

In [ ]:
top10bysize = suicides_2015[['Country','Suicides/100kPop']].groupby('Country').sum().sort_values("Suicides/100kPop", ascending=False)[0:10]

In [ ]:
top10bysize

In [ ]:
# Creating an intersection by index to see which countries appears in both rankings.
idx1 = pd.Index(top10bysize.index)
idx2 = pd.Index(top10countriesbynum.index)

In [ ]:
set_intersection

### MALE: TOP 10 COUNTRIES BY SIZE

In [ ]:
top10malesbysize = suicides_2015[['Country','Gender','Suicides/100kPop']].loc[suicides_2015['Gender']=='male'].groupby('Country').sum().sort_values("Suicides/100kPop", ascending=False)[0:10]

In [ ]:
top10malesbysize

### FEMALE: TOP 10 COUNTRIES BY SIZE

In [ ]:
top10femalesbysize = suicides_2015[['Country','Gender','Suicides/100kPop']].loc[suicides_2015['Gender']=='female'].groupby('Country').sum().sort_values("Suicides/100kPop", ascending=False)[0:10]

In [ ]:
top10femalesbysize

In [ ]:
# Creating an intersection by index to see if the countries with a higher number of suicides 
# by size of the population are the same countries for both genres.

idgsf = pd.Index(top10femalesbysize.index)
idgsm = pd.Index(top10malesbysize.index)

In [ ]:
set_intersection = idgsf.intersection(idgsm)

In [ ]:
set_intersection

In [ ]:
# females but no males:

idgsf.difference(idgsm)

In [ ]:
# males but not females:

idgsm.difference(idgsf)

### In this case, the countries Republic of Korea,  Lithuania, Hungary, Croatia, Slovenia and Uruguay match for both genres.

### But Belgiun, Iceland, Japan and Serbia appear in the top 10 for females while Kazakhstan, Latvia, Russian Federation and Ukraine appear in the top 10 for males.

# QUESTION 5: 

# 1.  Which generation has committed suicide the most/least?


## First, we will see which generation committed more suicides from 1990 till 2015:

## BY NUMBER:

In [ ]:
df_1 = suicides[['Generation','Num_Suicides']].groupby('Generation').sum().sort_values('Num_Suicides', ascending = False)

In [ ]:
df_1

### We see that in number of suicides, 'Boomers Generation' is the generation than committed the highest number of suicides. What will happen if we divide the number of suicides per generation by the total population for each generation? Let's check:

In [ ]:
df_2 = suicides[['Generation','Num_Suicides', 'Suicides/100kPop']].groupby('Generation').sum().sort_values('Suicides/100kPop', ascending = False)

In [ ]:
df_2 = df_2.round({'Suicides/100kPop': 2})

In [ ]:
df_2

# in 2015:

In [ ]:
# checking the data 

suicides_2015.head()

In [ ]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

### Let's see which generation committes more suicides in number of suicides:

In [ ]:
suicides_15 = suicides_2015[['Generation','Num_Suicides', 'Population']]

In [ ]:
suicides_15.groupby(['Generation']).sum().sort_values('Num_Suicides', ascending = False)

### We see that Silent and Z Generations commit the less suicides. This make sense because Silent Generation is the oldest one and generation Z is the youngest one so, probably, the population is lower than other range of ages. Let's calculate Suicide per generation divided by the total population:

In [ ]:
suicides15 =suicides_15.groupby(['Generation']).sum()

In [ ]:
suicides15['Sui/Pop'] = (suicides15['Num_Suicides']/suicides15['Population'])*1000000

In [ ]:
suicides15.sort_values('Sui/Pop', ascending = False)

### In terms of population, we see that Silent Generation committes more suicides followed by the Boomer generation.

# QUESTION 6:

# Countries with the highest 5-14 years range suicide rate?

## From 1990 to 2015:

In [ ]:
bbs = suicides[suicides['Age'] == '5-14 years']

In [ ]:
bbs = bbs[['Country','Suicides/100kPop', 'Population','Num_Suicides']].groupby('Country').sum().sort_values('Suicides/100kPop', ascending = False)

In [ ]:
bbs.head()

In [ ]:
bbs['Rate'] = bbs['Num_Suicides']/bbs['Suicides/100kPop']

In [ ]:
bbs.sort_values('Rate', ascending = False)[0:10].head()

# POR QUE ME DA DIFERENTE???

# Countries with the highest 5-14 years range suicide rate in 2015?

In [ ]:
bbs15 = suicides.loc[(suicides['Age'] == '5-14 years') & (suicides['Year'] == 2015)]

In [ ]:
bbs15 =  bbs15[['Country','Num_Suicides','Population', 'Suicides/100kPop']].groupby('Country').sum().sort_values('Suicides/100kPop', ascending = False)


In [ ]:
bbs15

In [ ]:
bbs15 = bbs15.rename(columns={"Population": "Pop_5to14"})

In [ ]:
bbs15['Rate'] = bbs15['Num_Suicides']/bbs15['Pop_5to14']

In [ ]:
bbs15 = bbs15.sort_values('Rate', ascending = False)[0:10]

In [ ]:
bbs15

# POR QUE ME DA DIFERENTE???

# QUESTION 6 Suicide/Population rate per country?

### TOP10 COUNTRIES WITH HIGHER RATE OF SUICIDES FROM 1990-2015:

In [ ]:
s_ratecountry = suicides[['Country','Num_Suicides','Population']].groupby('Country').sum()

In [ ]:
s_ratecountry['Rate'] = (s_ratecountry['Num_Suicides']/s_ratecountry['Population'])*1000000

In [ ]:
s_ratecountry.sort_values('Rate', ascending = False)[0:10]

### TOP10 COUNTRIES WITH HIGHER RATE OF SUICIDES IN 2015:

In [ ]:
s_ratecountry15 = suicides[suicides['Year'] == 2015]

In [ ]:
s_ratecountry15 = s_ratecountry15[['Country','Num_Suicides','Population']].groupby('Country').sum()

In [ ]:
s_ratecountry15['Rate'] = (s_ratecountry15['Num_Suicides']/s_ratecountry15['Population'])*1000000

In [ ]:
s_ratecountry15.sort_values('Rate', ascending = False)[0:10]

In [ ]:
df1 = suicides_9015.groupby('Generation').sum().sort_values("Num_Suicides", ascending=False)

In [ ]:
df1['Gen_sui/tot_sui'] = df1['Num_Suicides'].apply(lambda row: (row/sum(df1['Num_Suicides'])*100))

In [ ]:
df1_f = df1[['Num_Suicides','Gen_sui/tot_sui']].round({'Gen_sui/tot_sui': 2})

In [ ]:
pop_gen = suicides_9015[['Generation','Population']].groupby('Generation').sum().sort_values('Population', ascending = False)

In [ ]:
pop_gen[''] = df1['Num_Suicides'].apply(lambda row: (row/sum(df1['Num_Suicides'])*100))

In [ ]:
df2 = df.groupby('Generation').sum().sort_values("Num_Suicides", ascending=False)

In [ ]:
df2 = df2[['Population', 'Suicides/100kPop']]

In [ ]:
df2['Population'].sum()

In [ ]:
suicides.head()

In [ ]:
suicides_lrange = suicides

In [ ]:
suicides.groupby('Year')['Country'].unique().apply(lambda x: len(x))

In [ ]:
suicides.groupby(['Country','Year']).sum().loc[('Spain',2012)]